# Detector de idiomas con Inteligencia Artificial integrada

Se va a diseñar un detector de idiomas utilizando recursos de **Machine Learning**. 
La idea principal es centrarlo en un problema de clasificación, con el que vamos a ayudarnos de una potente herramienta de ML para Python, **Scikit-Learn**.

In [14]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from time import time
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
from xgboost import XGBClassifier
import lightgbm as lgb
import pickle

Vamos a trabajar con corpus del Parlamento Europeo que se encuentran disponibles en el siguiente enlace:
http://www.statmt.org/europarl/

Abrimos los diferentes corpus.
Aclaramos los idiomas:

it = italiano  
es = español  
en = inglés  
pt = portugués  
fr = francés

In [2]:
import os
notebook_dir = os.getcwd()
archivo_italiano = os.path.join(notebook_dir, 'corpus', 'italiano')

In [6]:
import os

ruta_base = r'C:\Users\themu\Desktop\corpus'

archivo_italiano = os.path.join(ruta_base, 'italiano')

In [8]:
if os.path.exists(archivo_italiano):
    with open(archivo_italiano, 'r', encoding='utf-8') as f:
        contenido = f.read()
    print(contenido[:100])  
else:
    print(f"El archivo {archivo_italiano} no existe.")

Ripresa della sessione
Dichiaro ripresa la sessione del Parlamento europeo, interrotta venerdì 17 di


In [10]:
import os

ruta_base = r'C:\Users\themu\Desktop\corpus'

archivos = ['italiano', 'espanol', 'ingles', 'frances', 'portugues']

file_handles = []

for archivo in archivos:
    ruta_completa = os.path.join(ruta_base, archivo)
    try:
        f = open(ruta_completa, 'r', encoding='utf8')
        file_handles.append(f)
    except FileNotFoundError:
        print(f"No se pudo encontrar el archivo {ruta_completa}")

# Por ejemplo:
it = file_handles[0]  # italiano
es = file_handles[1]  # espanol
en = file_handles[2]  # ingles
fr = file_handles[3]  # frances
pt = file_handles[4]  # portugues

# Ejemplo de uso: leer las primeras líneas de italiano
if it:
    contenido_italiano = it.read()
    print(contenido_italiano[:100])  # Imprime los primeros 100 caracteres del archivo italiano
else:
    print("No se pudo abrir el archivo italiano.")

for f in file_handles:
    f.close()

Ripresa della sessione
Dichiaro ripresa la sessione del Parlamento europeo, interrotta venerdì 17 di


In [65]:
corpus_list = ['it', 'es', 'en', 'fr', 'pt']

Con la variable *size* le vamos a decir al programa la cantidad de frases con las que vamos a trabajar. Para que sea más fácil y todos los idiomas estén igualmente representados, procuraremos que *size* sea múltimo de 5

In [67]:
size = 10000*5

A continuación vamos a crear el DataFrame que estará constituido simplemente con dos columnas, la primera será una frase y la segunda el idioma de dicha frase.

In [12]:
from time import time
import pandas as pd

corpus_list = ['italiano', 'espanol', 'ingles', 'frances', 'portugues']

t0 = time()

leng = []
txt = []
size = 100  

file_handles = []

try:
    for archivo in corpus_list:
        ruta_completa = os.path.join(ruta_base, archivo)
        f = open(ruta_completa, 'r', encoding='utf8')
        file_handles.append(f)

    for f in file_handles:
        n = 0
        for linea in f:
            if n < (int(size / len(corpus_list))):
                if len(linea) >= 25:
                    n += 1
                    txt.append(linea[:-1])  # Elimina el último carácter de la línea (posiblemente '\n')
                    leng.append(archivo)   # Agrega el nombre del archivo como identificador de idioma
            else:
                break
finally:
    for f in file_handles:
        f.close()

df = pd.DataFrame()
df['texto'] = txt
df['idioma'] = leng

print("DataFrame creado en %0.4f s" % (time() - t0))

DataFrame creado en 0.1494 s


Podemos ver un pequeño resumen de nuestro DataFrame

In [94]:
df.describe()

,texto,idioma
count,100,100
unique,96,1
top,"Señora Presidenta, una cuestión de procedimiento.",portugues
freq,2,100


In [96]:
df[df.idioma=='es'].head()

,texto,idioma


In [98]:
df[df.idioma=='pt'].head()

,texto,idioma


Eliminamos las frases repetidas, ya que no son interesantes para este problema.

In [101]:
df.drop_duplicates(subset='texto', keep='first', inplace=True)

In [103]:
df.describe()

,texto,idioma
count,96,96
unique,96,1
top,Dichiaro ripresa la sessione del Parlamento eu...,portugues
freq,1,96


Crearemos, a partir del DataFrame, nuestro conjunto de entrenamiento, train, y nuestro conjunto de validación, test.
Para realizar esto utilizaremos la herramienta de Scikit-Learn *train_test_split* cuyo parámetro *test_size* nos va a indicar el tamaño del conjunto de validación.

In [106]:
X_train, X_test, y_train, y_test = train_test_split(
    df['texto'],
    df['idioma'],
    test_size=0.4,
    random_state=0
)

In [108]:
X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    test_size=0.5,
    random_state=0
)

In [110]:
X_train.shape, X_test.shape, X_val.shape

((57,), (19,), (20,))

Con la funcion *feature_extraction.text.TfidfVectorizer* podremos separar la frase por palabras, lo que se conoce como *tokenizar*. El parámetro *analyzer='word'* nos indica que vamos a separar por palabras.

In [113]:
n_fts = 10000
vectorizer = feature_extraction.text.TfidfVectorizer(
    analyzer = 'word',
    ngram_range = (1, 1),
    max_features = n_fts,
    strip_accents = 'ascii',
    max_df = 0.8,
#     min_df = 0.01
)

Entrenamos un Pipeline con el TfIdf y un algoritmo RandomForest

In [237]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=10,
    max_features='auto',
    max_samples=0.8,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    criterion='gini'
)

In [128]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Ejemplo de datos
corpus_list = ['italiano', 'espanol', 'ingles', 'frances', 'portugues']
X_train, X_test, y_train, y_test = train_test_split(corpus_list, corpus_list, test_size=0.2, random_state=42)

# Definir el vectorizador y el clasificador
vectorizer = TfidfVectorizer()
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=10,
    max_features='sqrt',  # Aquí se usa 'sqrt' en lugar de 'auto'
    max_samples=0.8,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    criterion='gini'
)

# Crear el modelo como un pipeline
model = Pipeline([
    ('vectorizer', vectorizer),
    ('model', clf)
])

# Entrenar el modelo
print('Entrenando modelo {}'.format(clf))
model.fit(X_train, y_train)


Entrenando modelo RandomForestClassifier(max_depth=10, max_samples=0.8, min_samples_leaf=10,
                       min_samples_split=5, n_estimators=200, n_jobs=-1,
                       oob_score=True, random_state=0)


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model',
                 RandomForestClassifier(max_depth=10, max_samples=0.8,
                                        min_samples_leaf=10,
                                        min_samples_split=5, n_estimators=200,
                                        n_jobs=-1, oob_score=True,
                                        random_state=0))])

Grado de acierto en los diferentes conjuntos

In [130]:
print('Acierto en train:')
preds_tr = model.predict(X_train)
acc = accuracy_score(y_train, preds_tr)
print(acc,'\n')

print('Acierto en test:')
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(acc,'\n')

print('Acierto en validación:')
preds_val = model.predict(X_val)
acc = accuracy_score(y_val, preds_val)
print(acc,'\n')

Acierto en train:
0.25 

Acierto en test:
0.0 

Acierto en validación:
0.0 



In [132]:
cm = confusion_matrix(y_test, preds, labels=corpus_list)

Podemos visualizar una matriz de confusión para hacer una primera comprobación tanto de los errores como de los aciertos cometidos por el modelo

In [134]:
pd.DataFrame(cm,index=corpus_list,columns=corpus_list)

,italiano,espanol,ingles,frances,portugues
italiano,0,0,0,0,0
espanol,0,0,1,0,0
ingles,0,0,0,0,0
frances,0,0,0,0,0
portugues,0,0,0,0,0


También podemos ver, a través de la función *classification_report* la precisión del modelo por cada idioma.

In [14]:
from sklearn.metrics import classification_report

# Obtener las etiquetas únicas presentes en y_test y preds
unique_labels = sorted(set(y_test).union(preds))

# Imprimir el reporte de clasificación usando labels
print(classification_report(y_test, preds, labels=unique_labels, target_names=corpus_list))



NameError: name 'y_test' is not defined

Ahora podemos guardar nuestras predicciones en un dataframe.

In [174]:
predictions = pd.DataFrame()
predictions['texto'] = X_test
predictions['idioma'] = preds
predictions[0:10]

,texto,idioma
0,espanol,ingles


Guardamos el modelo entrenado para poder ejecutarlo en la aplicación Flask

In [179]:
import os

# Verifica y crea el directorio 'models' si no existe
directory = 'models'
if not os.path.exists(directory):
    os.makedirs(directory)


In [187]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Suponiendo que 'vectorizer' y 'clf' están definidos previamente
vectorizer = TfidfVectorizer()
clf = RandomForestClassifier()

# Crear el modelo como un pipeline
model = Pipeline([
    ('vectorizer', vectorizer),
    ('model', clf)
])

# Guardar el modelo en 'models/model.pkl'
joblib.dump(model, 'models/model.pkl', compress=1)


['models/model.pkl']

In [189]:
new_model = joblib.load('models/model.pkl')

A continuación vamos a crear una función sencilla, lo que sería el detector en sí, donde el parámetro de entrada será una frase y la salida será el idioma de dicha frase.

In [192]:
corpus_list = ['it', 'es', 'en', 'fr', 'pt']

In [204]:
def detection(model, x):
    lenguage=model.predict([x])[0]
    proba = model.predict_proba([x])
    print(proba, max(proba[0]))
    if lenguage == 'fr':
        lenguage = 'francés'
    elif lenguage == 'pt':
        lenguage = 'portugués'
    elif lenguage == 'it':
        lenguage = 'italiano'
    elif lenguage == 'en':
        lenguage = 'inglés'
    elif lenguage == 'es':
        lenguage = 'español'
        
    print('El idioma de la frase es: '+lenguage)

Evaluemos algunos ejemplos:

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import joblib

# Suponiendo que 'vectorizer' y 'clf' están definidos previamente
vectorizer = TfidfVectorizer()
clf = RandomForestClassifier()

# Crear el modelo como un pipeline
model = Pipeline([
    ('vectorizer', vectorizer),
    ('model', clf)
])

# Definir datos de entrenamiento
X_train = [
    'ejemplo de texto 1',
    'ejemplo de texto 2',
    'ejemplo de texto 3'
]
y_train = ['etiqueta1', 'etiqueta2', 'etiqueta3']

# Ajustar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Función para detección de idioma
def detection(model, x):
    # Aquí se asume que 'model' ya está ajustado correctamente
    language = model.predict([x])[0]
    proba = model.predict_proba([x])
    print(proba, max(proba[0]))

# Ejemplo de uso de la función 'detection'
detection(model, 'hola, mi nombre es Multinecz')

# Guardar el modelo ajustado
joblib.dump(model, 'models/model.pkl', compress=1)

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


['models/model.pkl']

In [28]:
import joblib

joblib.dump(model, 'models/model.pkl', compress=1)

['models/model.pkl']

In [30]:
import joblib

# Cargar el modelo guardado
new_model = joblib.load('models/model.pkl')

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Verificar si el vectorizador está ajustado
if not isinstance(new_model.named_steps['vectorizer'], TfidfVectorizer):
    raise ValueError("El vectorizador TF-IDF no está correctamente definido en el modelo.")

# Llamar a la función 'detection'
detection(new_model, 'hola, mi nombre es Nícolas')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


In [36]:
detection(new_model, 'suscríbete al canal para seguir aprendiendo')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


In [38]:
detection(new_model, 'subscribe and like to continue learning')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


In [40]:
detection(new_model, 'hello, my name is Nícolas')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


In [42]:
detection(new_model, 'Alguem aqui fala espanhol?')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335


In [44]:
detection(new_model, 'Piacere di conoscerla')

[[0.32       0.30666667 0.37333333]] 0.3733333333333335
